In [3]:
### This script looks at all files
### 1) Checks deleted files for md5 related fields and clears them
###                         for qc metric and changes status of qc metric object to deleted
###                         clears the qc metric field
###                         for workflows and deletes all of them
### 2) Checks other files for workflows and deleted old workflows
###                                     and deleted problematic ones (status or rev)
### takes around 20 min

from core.utils import Tibanna
from core import ff_utils
from datetime import datetime

# set enviroment and key/connection
env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

def get_wfr_report(wfrs):
    wfr_report = []
    for wfr_data in wfrs:
        wfr_rep = {}
        """For a given workflow_run_sbg item, grabs details, uuid, run_status, wfr name, date, and run time"""
        wfr_uuid = wfr_data['uuid']
        wfr_data = ff_utils.get_metadata(wfr_uuid, connection = ff)
        wfr_status = wfr_data['run_status']
        try:
            wfr_name = wfr_data['title'].split(' run ')[0]
        except:
            print('ProblematicCase')
            print(wfr_data['uuid'], wfr_data.get('display_title', 'no title'))
            continue
        wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
        run_hours = (datetime.utcnow()-wfr_time).total_seconds()/3600
        wfr_name_list = wfr_data['title'].split(' run ')[0].split('/')
        wfr_name = wfr_name_list[0]
        try:
            wfr_rev = wfr_name_list[1] 
        except:
            wfr_rev = "0"

        output_files = wfr_data.get('output_files',None)
        output_uuids = []
        if output_files:
            for i in output_files:
                if i.get('value', None):
                    output_uuids.append(i['value'])

        wfr_rep = {'wfr_uuid': wfr_data['uuid'],
                   'wfr_status': wfr_data['run_status'],
                   'wfr_name': wfr_name,
                   'wfr_rev': wfr_rev,
                   'wfr_date': wfr_time,
                   'run_time': run_hours,
                   'status': wfr_data['status'],
                   'outputs': output_uuids}
        wfr_report.append(wfr_rep)
    wfr_report = sorted(wfr_report, key=lambda k: (k['wfr_date'], k['wfr_name']))
    return wfr_report

    
def printTable(myDict, colList=None):
    """ Pretty print a list of dictionaries Author: Thierry Husson"""
    if not colList: colList = list(myDict[0].keys() if myDict else [])
    myList = [colList] # 1st row = header
    for item in myDict: myList.append([str(item[col] or '') for col in colList])
    colSize = [max(map(len,col)) for col in zip(*myList)]
    formatStr = ' | '.join(["{{:<{}}}".format(i) for i in colSize])
    myList.insert(1, ['-' * i for i in colSize]) # Seperating line
    for item in myList: print(formatStr.format(*item))
        


In [5]:
print 'started at', datetime.utcnow()
delete_workflows = raw_input("Do you want to delete old workflowruns (if not, only report will be displayed (y/n))")

# what kind of files should be searched for worflow run inputs, use url compatible naming

# accepted workflows
# workflow name, accepted revision numbers (0 if none), accetable run time (hours)
workflow_details = [
                    ['md5', ['0'], 24],
                    ['fastqc-0-11-4-1', ['0', '1'], 24],
                    ['hi-c-processing-parta-juicer', ['25','26'], 150],
                    ['hi-c-processing-partb', ['31', '34', '38'],150],
                    ['hi-c-processing-partc', ['3', '8'], 150],
                    ['add-hic-normvector-to-mcool', ['3'], 150],
                    ['extract-mcool-normvector-for-juicebox', ['100'], 150],
                    ['extract-mcool-normvector-for-juicebox-1', ['1'], 150],
                    ['bwa-mem', ['0'], 150],
                    ['pairsam-parse-sort',['0'],150],
                    ['pairsam-merge',['0'],150],
                    ['pairsam-markasdup',['0'],150],
                    ['pairsam-filter',['0'],150],
                    ['addfragtopairs',['0'],150],
                    ['pairs-patch',['0'],150],
                    ['hi-c-processing-partb set',['0'],150],
                    ['hi-c-processing-partb exp',['0'],150],
                    ['hi-c-processing-partc set',['0'],150],
                    ['hi-c-processing-partc exp',['0'],150]
                   ]
workflow_names = [i[0] for i in workflow_details]

deleted_wfr_no = 0
files_with_deleted_wfr = 0
files = [i['uuid'] for i in ff_utils.get_metadata('files-fastq' , connection=ff)['@graph']]
print len(files), 'files in the system'
deleted_wfrs = []
counter = 0
del_md5 = 0
del_qc = 0
deleted_output = 0
for a_file in files:
    counter += 1
    if counter % 100 == 0:
        print counter, files_with_deleted_wfr
    raw_file = ff_utils.get_metadata(a_file, connection = ff, frame='embedded')
    deleted_wf = False
    wfr_report = []
    wfrs = raw_file.get('workflow_run_inputs')
    
    # Delete wfrs if file is deleted
    if raw_file['status'] == 'deleted':
        if delete_workflows.lower() in ['y', 'yes']:
            # clean deleted files of md5 and qc metrics
            for a_field in ['content_md5sum', 'md5sum']:  
                if raw_file.get(a_field):
                    ff_utils.delete_field(raw_file, a_field, connection=ff)
                    del_md5 += 1
            if raw_file.get('quality_metric'):
                qc_uuid = raw_file['quality_metric']['uuid']
                ff_utils.delete_field(raw_file, 'quality_metric', connection=ff)
                # delete quality metrics object
                patch_data = {'status': "deleted"}
                ff_utils.patch_metadata(patch_data, obj_id=qc_uuid ,connection=ff)
                del_qc += 1
        # delete all workflows for deleted files
        if not wfrs:
            continue
        else:
            wfr_report = get_wfr_report(wfrs)
            for wfr_to_del in wfr_report:
                if wfr_to_del['status'] != 'deleted':
                    if wfr_to_del['wfr_name'] not in workflow_names:
                        print('Unlisted Workflow', wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                    deleted_wf = True
                    deleted_wfr_no += 1
                    
                    ####################################################
                    ## TEMPORARY PIECE##################################
                    if wfr_to_del['status'] == 'released to project':
                        print('saved from deletion', wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                        continue
                    if wfr_to_del['status'] == 'released':
                        print('delete released!!!!!', wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                        continue  
                    #####################################################
                    
                    print(wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                    if delete_workflows.lower() in ['y', 'yes']:
                        patch_data = {'description': "This workflow run is deleted", 'status': "deleted"}
                        deleted_wfrs.append(wfr_to_del['wfr_uuid'])
                        ff_utils.patch_metadata(patch_data, obj_id=wfr_to_del['wfr_uuid'] ,connection=ff)
                        # delete output files of the deleted workflow run
                        if wfr_to_del['outputs']:
                            for out_file in wfr_to_del['outputs']:
                                deleted_output += 1
                                ff_utils.patch_metadata({'status': "deleted"}, obj_id=out_file ,connection=ff)
       
                
    else:
        # get a report on all workflow_runs
        if not wfrs:
            continue
        else:
            wfr_report = get_wfr_report(wfrs)
            # printTable(wfr_report, ['wfr_name', 'run_time', 'wfr_rev', 'run_time', 'wfr_status'])
            
            # check if any unlisted wfr in report
            my_wfr_names = [i['wfr_name'] for i in wfr_report]
            unlisted = [x for x in my_wfr_names if x not in workflow_names]
            if unlisted:
                print('Unlisted Workflow', unlisted, 'skipped in', raw_file['accession'])
                    
            for wf_name,accepted_rev,accepted_run_time in workflow_details:
                #for each type of worklow make a list of old ones, and patch status and description
                sub_wfrs = [i for i in wfr_report if i['wfr_name'] == wf_name]
                if sub_wfrs:
                    active_wfr = sub_wfrs[-1]
                    old_wfrs = sub_wfrs [:-1]
                    # check the status of the most recent workflow
                    if active_wfr['wfr_status'] != 'complete':
                        if active_wfr['wfr_status'] in ['running', 'started'] and active_wfr['run_time'] < accepted_run_time:
                            print wf_name,'still running for', a_file
                        else:
                            old_wfrs.append(active_wfr)
                    elif active_wfr['wfr_rev'] not in accepted_rev:
                        old_wfrs.append(active_wfr)
                    if old_wfrs:
                        for wfr_to_del in old_wfrs:
                            if wfr_to_del['status'] != 'deleted':
                                deleted_wf = True
                                deleted_wfr_no += 1 
                                
                                ####################################################
                                ## TEMPORARY PIECE
                                if wfr_to_del['status'] == 'released to project':
                                    print('saved from deletion',wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                    continue
                                if wfr_to_del['status'] == 'released':
                                    print('delete released????',wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                    continue
                                ####################################################

                                print(wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                
                                if delete_workflows.lower() in ['y', 'yes']:
                                    patch_data = {'description': "This workflow run is deleted", 'status': "deleted"}
                                    deleted_wfrs.append(wfr_to_del['wfr_uuid'])
                                    
                                    ff_utils.patch_metadata(patch_data, obj_id=wfr_to_del['wfr_uuid'] ,connection=ff)
                                    # delete output files of the deleted workflow run
                                    if wfr_to_del['outputs']:
                                        for out_file in wfr_to_del['outputs']:
                                            deleted_output += 1
                                            ff_utils.patch_metadata({'status': "deleted"}, obj_id=out_file ,connection=ff)
    if deleted_wf:
        files_with_deleted_wfr += 1

if delete_workflows.lower() in ['y', 'yes']:
    print str(deleted_wfr_no),"workflowruns from", str(files_with_deleted_wfr), "files deleted"
else:
    print str(deleted_wfr_no),"workflowruns from", str(files_with_deleted_wfr), "files need to be deleted"

print len(deleted_wfrs)
print del_md5, 'md5 fields deleted'
print del_qc, 'qc metrics deleted'
print deleted_output, 'deleted output files'
print 'finished at', datetime.utcnow()

started at 2018-02-13 17:37:16.468214
Do you want to delete old workflowruns (if not, only report will be displayed (y/n))y
3589 files in the system
(u'md5', 'old style or dub', u'47615e8a-08c9-40da-93e2-0a3998cb53d6', u'4DNFIN1LV6LU')
(u'md5', 'old style or dub', u'c96091a4-f408-4396-9a93-fbf5f62ba17f', u'4DNFITGLGFJD')
(u'md5', 'old style or dub', u'05527278-2af6-40ed-b3ef-55c224f499a3', u'4DNFIGJ68IX1')
(u'md5', 'old style or dub', u'92c600a4-3996-48eb-9bd6-314395ece4be', u'4DNFIP6B1YN6')
(u'md5', 'old style or dub', u'44386663-d965-49f8-8110-e9c2d6418ef9', u'4DNFIALTLJL8')
(u'md5', 'old style or dub', u'c8127ad5-bd4a-40ab-8fdd-52a3c00d2f6d', u'4DNFI7QGWPXP')
(u'md5', 'old style or dub', u'ff0f3bcf-b7ca-4fe0-9c7e-ac821fd07f89', u'4DNFI6727ZM2')
(u'md5', 'old style or dub', u'66ef704a-47df-4f7f-8db1-69fa47c2ef25', u'4DNFIX15AHHH')
(u'md5', 'old style or dub', u'a33314c6-3d4c-4223-9d69-36a9753adac3', u'4DNFIBQP5BC1')
(u'md5', 'old style or dub', u'6281256d-09c1-4a96-9733-d6d346a4ff7f'

(u'fastqc-0-11-4-1', 'old style or dub', u'dd6c52d9-1cea-4d1b-82ca-0d2fc195ae22', u'4DNFIO5IUBAD')
(u'fastqc-0-11-4-1', 'old style or dub', u'f53972a9-7613-4e5a-aced-9c36d9afaeaa', u'4DNFIXOYA8T6')
(u'fastqc-0-11-4-1', 'old style or dub', u'486896ff-62a4-4fc3-b98e-efeb3ce064b9', u'4DNFIXOYA8T6')
(u'fastqc-0-11-4-1', 'old style or dub', u'0cd1bdeb-3989-4e43-b749-7fbe94e37c32', u'4DNFIPQX8MM8')
(u'fastqc-0-11-4-1', 'old style or dub', u'9d5baa17-0ed9-45c9-9aa8-7264a8e3ec99', u'4DNFIPQX8MM8')
(u'fastqc-0-11-4-1', 'old style or dub', u'6ac07e29-1fe6-4ff2-9c5e-40a360c73252', u'4DNFIF5ZRDYC')
(u'fastqc-0-11-4-1', 'old style or dub', u'ec370ad2-ab7e-497c-9757-32fbfbd7e321', u'4DNFIF5ZRDYC')
(u'fastqc-0-11-4-1', 'old style or dub', u'34739100-acb2-400c-aa5d-ecb7ac0659e8', u'4DNFI61TSIX2')
(u'fastqc-0-11-4-1', 'old style or dub', u'da0416f9-dc3b-4796-aa36-776c791a8c4d', u'4DNFI61TSIX2')
(u'fastqc-0-11-4-1', 'old style or dub', u'6456de63-22bd-44cb-8d88-c3bb4f7dd0c4', u'4DNFIWIWCZA2')
(u'fastqc-

(u'fastqc-0-11-4-1', 'old style or dub', u'd54c342c-f680-4d96-82d9-701cfb43d976', u'4DNFI78RZF3H')
(u'fastqc-0-11-4-1', 'old style or dub', u'8d3f1751-ace7-4378-a279-d9efa989b1a8', u'4DNFI78RZF3H')
(u'fastqc-0-11-4-1', 'old style or dub', u'47b4574d-57cd-46e0-934b-037d49e08ad7', u'4DNFI24DAINB')
(u'fastqc-0-11-4-1', 'old style or dub', u'285eb320-f1fe-4e58-a780-265823c481b0', u'4DNFI24DAINB')
(u'fastqc-0-11-4-1', 'old style or dub', u'4e9254fa-e647-4e45-9a7b-76c0261d0926', u'4DNFIJB49KS9')
(u'fastqc-0-11-4-1', 'old style or dub', u'399c7efe-b9d7-493b-a0ac-193f56a93fff', u'4DNFIJB49KS9')
(u'fastqc-0-11-4-1', 'old style or dub', u'1f538b9b-dd5b-4902-b8a4-f5d8cee91f12', u'4DNFI91YZTQN')
(u'fastqc-0-11-4-1', 'old style or dub', u'cffff9f7-bfad-48ee-81d2-49d4e50fa852', u'4DNFI91YZTQN')
(u'fastqc-0-11-4-1', 'old style or dub', u'f97a8be5-d525-4b9e-9013-1c602c2ced3d', u'4DNFIDRUF8TL')
(u'fastqc-0-11-4-1', 'old style or dub', u'c8749a8c-cc8d-451b-a950-5a62790c3a5e', u'4DNFIDRUF8TL')
(u'fastqc-

(u'fastqc-0-11-4-1', 'old style or dub', u'322439de-5a42-4b25-b2c7-e75f0f5157d0', u'4DNFINHSE1C6')
(u'fastqc-0-11-4-1', 'old style or dub', u'70e1366b-77d5-4038-8e4a-7bd745a18054', u'4DNFII6I1GON')
(u'fastqc-0-11-4-1', 'old style or dub', u'dc05598b-09ef-4589-b4a7-8f3d48374e75', u'4DNFII6I1GON')
(u'fastqc-0-11-4-1', 'old style or dub', u'a43b2c19-bb5b-438e-92b9-4c08eb84878e', u'4DNFIDQ1WH8W')
(u'fastqc-0-11-4-1', 'old style or dub', u'fef4c775-32ba-460f-9099-17263300c56c', u'4DNFIDQ1WH8W')
(u'fastqc-0-11-4-1', 'old style or dub', u'c6fd7513-1cd8-4fb2-af65-c9d6ede24b45', u'4DNFIKMCPMKP')
(u'fastqc-0-11-4-1', 'old style or dub', u'04ac45f4-0fb5-4065-8c7a-b0f41dc048db', u'4DNFIKMCPMKP')
(u'fastqc-0-11-4-1', 'old style or dub', u'b77f6abd-f2c9-4cc9-ab99-36896d11a07e', u'4DNFI3QVF5AV')
(u'fastqc-0-11-4-1', 'old style or dub', u'd7eaede3-daff-4be9-a0b7-91fa950790e5', u'4DNFI3QVF5AV')
400 148
(u'fastqc-0-11-4-1', 'old style or dub', u'59fdae89-3ea0-48c8-af1d-dd6b8bf454c6', u'4DNFIIZDUI3Q')
(u

(u'fastqc-0-11-4-1', 'old style or dub', u'8f1526b4-61f0-4c94-8acb-5aeef10c38ea', u'4DNFIUHA2GNC')
(u'fastqc-0-11-4-1', 'old style or dub', u'9e5e25f4-954b-4492-9a66-20e4de39e40a', u'4DNFIUHA2GNC')
(u'fastqc-0-11-4-1', 'old style or dub', u'564fd754-cca1-4092-b0be-1097d5fded5e', u'4DNFICBHO5SD')
(u'fastqc-0-11-4-1', 'old style or dub', u'c6801c98-5d47-46c2-8aef-39876b927a6f', u'4DNFICBHO5SD')
(u'fastqc-0-11-4-1', 'old style or dub', u'b2f8d2ec-02bc-411f-93f2-82c26b49987b', u'4DNFI5NBPBTJ')
(u'fastqc-0-11-4-1', 'old style or dub', u'c1bf7a5e-2d84-4c8c-b427-0e7f2f2778d3', u'4DNFI5NBPBTJ')
(u'fastqc-0-11-4-1', 'old style or dub', u'e82c886b-9f42-4b07-ab1e-9ea9011b12af', u'4DNFIQBQZMQS')
(u'fastqc-0-11-4-1', 'old style or dub', u'e9110eae-8d84-4d2a-a1be-21e7d166a03b', u'4DNFIQBQZMQS')
(u'fastqc-0-11-4-1', 'old style or dub', u'6c6698c5-8f02-4758-87e6-288548a07ab8', u'4DNFI9P54Q8N')
(u'fastqc-0-11-4-1', 'old style or dub', u'255f9314-8924-44f4-9431-619106a44e80', u'4DNFI9P54Q8N')
(u'fastqc-

ValueError: No JSON object could be decoded